In [1]:
# data
import hashlib
import pandas as pd
import numpy as np

# parameter tuning
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

# model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from joblib import dump, load

# sampling
from scipy.stats import uniform, randint

# plots
import matplotlib.pyplot as plt

# custom objects 
from production_demo import CategoriesTransformer 

ModuleNotFoundError: No module named 'production_demo'

### Dataprep

In [ ]:
train = pd.read_csv('../data/train.csv')


# select model columns
id_col = 'Id'
output = 'SalePrice'
numerics = [col for col in train.select_dtypes(np.number).columns 
            if col not in [id_col, output]]
categories = [col for col in train.select_dtypes('object').columns 
              if col not in [id_col, output]]


# hasher 
hct = CategoriesTransformer(categories)


# prepare train/test splitting
train.sort_values(by=['YrSold', 'MoSold'], 
                  inplace=True)
tss = TimeSeriesSplit(n_splits=10)


# parameter space
param_distributions = dict(
    LGBM__num_leaves=randint(1, 5000),
    LGBM__max_depth=randint(1, 20),
    LGBM__learning_rate=uniform(0.01, 0.9),
    LGBM__n_estimators=randint(5, 1000),
    LGBM__min_split_gain=uniform(0.0, 0.1),
    LGBM__min_child_weight=uniform(0.0, 0.1),
    LGBM__subsample=uniform(0.1, 0.9),
    LGBM__colsample_bytree=uniform(0.1, 0.9),
    LGBM__reg_alpha=uniform(0.0, 5000.0),
    LGBM__reg_lambda=uniform(0.0, 5000.0),
)

### Param tuning

In [ ]:
model = Pipeline([
    ('hash', hct),
    ('LGBM', LGBMRegressor(random_state=22)),
])
rsv = RandomizedSearchCV(estimator=model,
                         param_distributions=param_distributions,
                         n_iter=1000,
                         cv=tss,
                         scoring='neg_root_mean_squared_error')
_ = rsv.fit(train[numerics + categories], train[output])

In [ ]:
best_params_dict = rsv.best_params_

print(f'Best params:\n {best_params_dict}')
print(f'\nBest score:\n {rsv.best_score_:.4f}')
model = LGBMRegressor(**best_params_dict)

## Train

In [ ]:
model.fit(train[numerics], train[output])

In [ ]:
#save model artifacts
dump(model, '../data/trained_model')

In [ ]:
!ls ../data